In [ ]:
# импортируем библиотеки для парсинга и обработки информации
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# Вводится путь к вебдрайверу, который был превращен в специальный объект Service
PATH = b"C:\Users\nastya\Desktop\Google Chrome\chromedriver.exe"
# PATH = input('Введите путь к вебдрайверу')
service = Service(PATH)
# запускаем браузер с помощью веб-драйвера
browser = webdriver.Chrome(service=service)
# открываем сайт, который будем пасрить
browser.get('https://www.eldorado.ru/c/smartfony/')
# дадим сайту время на прогрузку динамических элементов
time.sleep(3)

# передадим в range 35, чтоб нажать кнопку "Показать еще" 35 раз, если больше, то парсер будет работать часами
for i in range(35):
    try:
        # находим и нажимаем кнопку 
        browser.find_element(By.CSS_SELECTOR, value='#listing-container > div.tq > button').click() 
        # немного ждем
        time.sleep(1.5)
    # когда такая копка исчезнет, выйдем из цикла
    except NoSuchElementException:
        break
    
# сохраним получившийся html код страницы для дальнейшей обработки
html = browser.page_source    
# Обработает html код с помощью BeautifulSoup и парсерса lxml 
soup = BeautifulSoup(html, 'lxml')
# соберем элементы, где находится основная информация по телефонам,чтоб достать ссылки на индивидуальные страницы
phones = soup.find_all('li', {'class': 'kG'})

# ключами этого словаря являются названия моделей, значениями - другие словари, в которых будут хранится их характеристики
# достаем ссылки и сразу записываем их как значения вложенного словаря, где ключом является название будущего столбца
info = {}
for phone in phones:
    phone_info = phone.find('div', {'class': 'lG nG'}).a
    info[phone_info.text] = {'link': 'https://www.eldorado.ru' + phone_info.get('href')}

In [ ]:
# переберем все ключи получившегося словаря, в котором уже есть ключ link и соответствующая ссылка
for model in info:
    # открываем сайт, который будем пасрить
    browser.get(info[model]['link'])
    # разворачиваем окно на весь экран 
    browser.fullscreen_window()
    # # немного ждем
    # time.sleep(1.5)
    
    
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
    # Скроллим вниз по странице
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # # Ждем, пока страницы прогрузится полностью
        # time.sleep(1.5)
        # Считает новую высоту скроллинга (положение страницы) со старой
        new_height = browser.execute_script("return document.body.scrollHeight")
        # если это конец страницы, перестает скроллить 
        if new_height == last_height:
            break
        last_height = new_height
        
    # нажимаем кнопку "Характеристики", чтоб увидеть все характеристики
    try:
        browser.find_element(By.XPATH, value='/html/body/div[11]/div[1]/div[2]/div[2]/div[4]/div/ul/li[2]/a').click()
        # time.sleep(1.5)
        
    
        # найдем таблицу со всем характеристиками и построчно будем считывать характеристики, добавляя их в словарь 
        chars_table = browser.find_element(By.XPATH, value='/html/body/div[11]/div[1]/div[2]/div[2]/div[6]/div/div[2]/table')
        chars_rows = chars_table.find_elements(By.TAG_NAME, value='tr')
        for char_row in chars_rows:
            try:
                char_info = char_row.find_elements(By.TAG_NAME, value='td')
                char_name = char_info[0].text
                char_value = char_info[1].text.split('\n')[0]
                info[model][char_name] = char_value
            except:
                pass
    except:
        pass

In [ ]:
# чтобы получился датафрейм, создаем новый словарь, 
# который будет принимать все признаки со всех моделей (они не у всех одни и те же) и проставлять пропуски там, 
# где признак у модели отсутствует, поэтому, возможно, будут столбцы с большим количеством пропусков
res_info = {}
for model in info:
    res_info[model] = {}
    for key in info[model].keys():
        res_info[model][key] = info[model].get(key)

In [ ]:
# создадим датафрейм по получившемуся словарю и сохраним таблицу в формате csv
df = pd.DataFrame(res_info)
df = df.T
df.to_csv('eldorado.csv')